# Football Data from Transfermarkt
- Kaggle Data URL

    - https://www.kaggle.com/datasets/davidcariboo/player-scores

## 데이터 설명
- Transfermarkt 웹사이트에서 스크레이핑한 각종 축구 데이터 (매주 한번 파일 업데이트)
    - 주요 대회 시즌별 60,000개 이상의 경기
    - 모든 대회의 400개 이상의 클럽들
    - 30,000 이상의 선수들
    - 400,000개 이상의 선수 가치 기록
    - 1,200,000개 이상의 선수 출전 기록

- `csv` 파일 설명
    - `apperances` : 선수 출장 기록
    - `club_games` : 클럽별 경기 홈팀, 어웨이팀 정보
    - `clubs` : 리그별 속해 있는 클럽
    - `competitions` : 대회 정보
    - `game_events` : 경기별 이벤트 정보 (카드, 득점, 어시스트 등)
    - `game_lineups` : 경기별 선수의 선발, 교체 명단 등재 여부
    - `games` : 경기에 대한 정보 (시즌, 라운드, 홈팀, 어웨이팀, 순위)
    - `player_valuations` : Transfermarkt 웹사이트에서 매긴 선수의 가치
    - `players` : 선수에 대한 세부 정보

- 데이터베이스 스키마
    <img src="https://raw.githubusercontent.com/dcaribou/transfermarkt-datasets/master/resources/diagram.svg?sanitize=true" width="1700">

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime
from datetime import timedelta
import missingno as msno

In [2]:
# 데이터 경로 지정
# C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data
apps_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/appearances.csv"
club_games_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/club_games.csv"
clubs_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/clubs.csv"
competitions_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/competitions.csv"
game_events_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/game_events.csv"
game_lineups_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/game_lineups.csv"
games_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/games.csv"
player_valuations_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/player_valuations.csv"
players_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/players.csv"

# 데이터셋 불러오기
apps_df = pd.read_csv(apps_path)
club_games_df = pd.read_csv(club_games_path)
clubs_df = pd.read_csv(clubs_path)
competitions_df = pd.read_csv(competitions_path)
game_events_df = pd.read_csv(game_events_path)
game_lineups_df = pd.read_csv(game_lineups_path)
games_df = pd.read_csv(games_path)
player_valuations_df = pd.read_csv(player_valuations_path)
players_df = pd.read_csv(players_path)

In [3]:
apps_df.head(3)

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45


In [4]:
club_games_df.head(3)

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
0,2320450,1468,0,NaN,Holger Bachthaler,24,2,NaN,Armin Veh,Home,0
1,2320460,1,3,NaN,Jürgen Luginger,86,1,NaN,Robin Dutt,Home,1
2,2320472,2036,4,NaN,Frank Schmidt,72,5,NaN,Alexander Schmidt,Home,0


In [5]:
clubs_df.tail(3)

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
423,800,atalanta-bergamo,Atalanta Bergamasca Calcio S.p.a.,IT1,NaN,25,26.5,17,68.0,16,Gewiss Stadium,21747,+€73.82m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/atalanta-berga...
424,979,moreirense-fc,Moreirense Futebol Clube,PO1,NaN,29,25.5,17,58.6,3,Estádio C. J. de Almeida Freitas,6153,€-2.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/moreirense-fc/...
425,984,west-bromwich-albion,West Bromwich Albion,GB1,NaN,25,27.8,8,32.0,4,The Hawthorns,26850,€-1.35m,NaN,2020,../data/raw/transfermarkt-scraper/2020/clubs.j...,https://www.transfermarkt.co.uk/west-bromwich-...


In [6]:
competitions_df.head(3)

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url
0,CIT,italy-cup,italy-cup,domestic_cup,domestic_cup,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/italy-cup/star...
1,NLSC,johan-cruijff-schaal,johan-cruijff-schaal,domestic_super_cup,other,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/johan-cruijff-...
2,GRP,kypello-elladas,kypello-elladas,domestic_cup,domestic_cup,56,Greece,GR1,europa,https://www.transfermarkt.co.uk/kypello-ellada...


In [7]:
game_events_df.head(3)

,game_event_id,date,game_id,minute,type,club_id,player_id,description,player_in_id,player_assist_id
0,2f41da30c471492e7d4a984951671677,2012-08-05,2211607,77,Cards,610,4425,"1. Yellow card , Mass confrontation",NaN,NaN
1,a72f7186d132775f234d3e2f7bc0ed5b,2012-08-05,2211607,77,Cards,383,33210,"1. Yellow card , Mass confrontation",NaN,NaN
2,b2d721eaed4692a5c59a92323689ef18,2012-08-05,2211607,3,Goals,383,36500,", Header, 1. Tournament Goal Assist: , Corner,...",NaN,56416.0


In [8]:
game_lineups_df.head(3)

,game_lineups_id,game_id,club_id,type,number,player_id,player_name,team_captain,position
0,b2dbe01c3656b06c8e23e9de714e26bb,2317258,610,substitutes,5,1443,Christian Poulsen,0,Defensive Midfield
1,b50a3ec6d52fd1490aab42042ac4f738,2317258,610,starting_lineup,4,5017,Niklas Moisander,0,Centre-Back
2,7d890e6d0ff8af84b065839966a0ec81,2317258,1090,substitutes,11,9602,Maarten Martens,0,Left Winger


In [9]:
games_df.head(3)

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321044,L1,2013,2. Matchday,2013-08-18,16,23,2,1,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
1,2321060,L1,2013,3. Matchday,2013-08-25,23,24,0,2,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
2,2321086,L1,2013,6. Matchday,2013-09-21,4,16,1,1,15.0,...,Max-Morlock-Stadion,50000.0,Knut Kircher,https://www.transfermarkt.co.uk/1-fc-nuremberg...,4-2-3-1,4-2-3-1,1.FC Nuremberg,Borussia Dortmund,1:1,domestic_league


In [10]:
player_valuations_df.head(3)

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,405973,2000-01-20,150000,3057,BE1
1,342216,2001-07-20,100000,1241,SC1
2,3132,2003-12-09,400000,126,TR1


In [11]:
players_df.head(3)

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0


### `.info()` 확인

In [13]:
# df.info() 확인

dfs = [apps_df, club_games_df, clubs_df, competitions_df, game_events_df, game_lineups_df, 
       games_df, player_valuations_df, players_df]

for df in dfs:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540638 entries, 0 to 1540637
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   appearance_id           1540638 non-null  object
 1   game_id                 1540638 non-null  int64 
 2   player_id               1540638 non-null  int64 
 3   player_club_id          1540638 non-null  int64 
 4   player_current_club_id  1540638 non-null  int64 
 5   date                    1540638 non-null  object
 6   player_name             1540637 non-null  object
 7   competition_id          1540638 non-null  object
 8   yellow_cards            1540638 non-null  int64 
 9   red_cards               1540638 non-null  int64 
 10  goals                   1540638 non-null  int64 
 11  assists                 1540638 non-null  int64 
 12  minutes_played          1540638 non-null  int64 
dtypes: int64(9), object(4)
memory usage: 152.8+ MB
None
<class 'pandas.core.

In [14]:
apps_copy = apps_df.copy()

### 사용 데이터셋
- 목표 : 선수의 시즌별 스탯 확인
    - `apperances` : 선수 출장 기록
    - `competitions` : 대회 정보
    - `players` : 선수에 대한 세부 정보
    
- 해볼 수 있는 것
    - 선수 스탯 예측
    - 선수 몸값 예측

In [15]:
apps_df.describe(include="all")

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
count,1540638,1.540638e+06,1.540638e+06,1.540638e+06,1.540638e+06,1540638,1540637,1540638,1.540638e+06,1.540638e+06,1.540638e+06,1.540638e+06,1.540638e+06
unique,1540638,NaN,NaN,NaN,NaN,3416,23472,43,NaN,NaN,NaN,NaN,NaN
top,2231978_38004,NaN,NaN,NaN,NaN,2020-10-04,Danilo,IT1,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,1795,1062,127333,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.045293e+06,1.855729e+05,2.967665e+03,3.642055e+03,NaN,NaN,NaN,1.486358e-01,3.806864e-03,9.627894e-02,7.582248e-02,6.956878e+01
std,NaN,5.774826e+05,1.704357e+05,7.534551e+03,9.289958e+03,NaN,NaN,NaN,3.670711e-01,6.158226e-02,3.316466e-01,2.862993e-01,2.979700e+01
min,NaN,2.211607e+06,1.000000e+01,1.000000e+00,-1.000000e+00,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,NaN,2.545933e+06,5.528200e+04,2.810000e+02,3.360000e+02,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.100000e+01
50%,NaN,2.981997e+06,1.299910e+05,8.260000e+02,9.310000e+02,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+01
75%,NaN,3.497453e+06,2.700510e+05,2.441000e+03,2.687000e+03,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+01


In [16]:
competitions_df.describe(include="all")

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url
count,43,43,43,43,43,43.000000,36,36,43,43
unique,43,42,42,11,4,NaN,14,14,1,43
top,CIT,premier-liga,premier-liga,first_tier,domestic_league,NaN,England,GB1,europa,https://www.transfermarkt.co.uk/italy-cup/star...
freq,1,2,2,14,14,NaN,4,4,43,1
mean,NaN,NaN,NaN,NaN,NaN,97.093023,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,69.766896,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,39.500000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,122.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,157.000000,NaN,NaN,NaN,NaN


In [17]:
players_df.describe()

,player_id,last_season,current_club_id,height_in_cm,market_value_in_eur,highest_market_value_in_eur
count,3.049500e+04,30495.000000,30495.000000,28410.000000,2.935200e+04,2.935200e+04
mean,3.142658e+05,2018.790589,4416.199115,182.250827,1.574004e+06,3.635859e+06
std,2.531242e+05,3.651044,10149.194332,6.841391,6.154743e+06,9.463744e+06
min,1.000000e+01,2012.000000,3.000000,18.000000,1.000000e+04,1.000000e+04
25%,9.598000e+04,2016.000000,403.000000,178.000000,1.000000e+05,2.750000e+05
50%,2.590250e+05,2020.000000,1075.000000,182.000000,2.500000e+05,8.000000e+05
75%,4.704080e+05,2022.000000,3057.000000,187.000000,6.500000e+05,2.800000e+06
max,1.229924e+06,2023.000000,83678.000000,207.000000,1.800000e+08,2.000000e+08


In [18]:
players_df.describe(include="object")

,first_name,last_name,name,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name
count,28505,30495,30495,30495,27773,28294,29919,30450,30321,30495,28122,19041,15115,30495,30495,30495,30495
unique,6596,22453,29847,29810,184,8227,182,8931,13,5,3,100,2671,24940,30495,14,424
top,David,García,Paulinho,paulinho,France,London,Spain,1996-01-19,Centre-Back,Defender,right,2024-06-30 00:00:00,Wasserman,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,TR1,Kilmarnock Football Club
freq,203,61,12,12,2185,434,1813,20,5357,9703,19733,5158,435,5556,1,3005,181


#### 사용 데이터셋 병합 함수 생성

In [75]:
# appearance_df, competitions_df, players_df merge 후 19-20 ~ 23-24 시즌 유럽 6대리그 데이터만 남김 (5시즌)
def app_comp_player_merge(apps_copy, competitions_df, players_df):
    # 데이터프레임의 date 컬럼 타입 datetime으로 변경
    apps_copy["date"] = pd.to_datetime(apps_copy["date"])
    
    # apps_copy와 competition_df competition_id를 기준으로 left join
    app_comp_df = pd.merge(apps_copy, competitions_df, how="left", on="competition_id")
    
    # 19-20 ~ 23-24 시즌 데이터만 남김 (5시즌)
    app_comp_df = app_comp_df[app_comp_df["date"]>="2019-06"]

    app_comp_player_df = pd.merge(app_comp_df, players_df, how="left", on="player_id")

#     cols = ['game_id', 'player_id', 'player_club_id',
#             'player_current_club_id', 'date', 'player_name', 'competition_id',
#             'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#             'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#             'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#             'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   
     
#     cols2 = ['player_id', 'player_club_id',
#              'player_current_club_id', 'date', 'player_name', 'competition_id',
#              'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#              'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#              'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#              'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   

    # 유럽 6대리그 (잉글랜드, 스페인, 이탈리아, 독일, 프랑스, 포르투갈 1부리그)만 필터링
    league_cond = ((app_comp_player_df.domestic_league_code=="GB1") | (app_comp_player_df.domestic_league_code=="ES1") | 
                   (app_comp_player_df.domestic_league_code=="IT1") | (app_comp_player_df.domestic_league_code=="L1") |
                   (app_comp_player_df.domestic_league_code=="FR1") | (app_comp_player_df.domestic_league_code=="PO1"))
#     app_comp_player_df = app_comp_player_df[cols2][league_cond].reset_index(drop=True)
    app_comp_player_df = app_comp_player_df[league_cond].reset_index(drop=True)

    # last_season 제대로 반영되지 않은 것들 발견하여, 새로 변경해준다
    # 6월을 넘지 않으면 직전 시즌 연도로 변경
    app_comp_player_df["last_season"] = app_comp_player_df["date"].apply(lambda x : x.year if x.month>=6 else x.year-1)

    # 수치형 변수 column 리스트
    group_cols = ["player_name", "last_season", "yellow_cards", "red_cards","goals", "assists", "minutes_played"]
    
    # groupby를 통해, 이름, 연도 기준으로 수치형 변수 합산 -> 시즌별 기록 파악하기 위함
    df_grouped = app_comp_player_df[group_cols].groupby(["player_name", "last_season"]).sum()
    df_grouped.reset_index(inplace=True)

#     # merge 하기 위한 기준인 "player_name", "last_season"은 남겨두고 group_cols와 일치하지 않는 컬럼 삭재
#     df_not_grouped = app_comp_player_df.loc[:, ~app_comp_player_df.columns.isin(group_cols[2:])]

#     # 위에서 만든 df_grouped, df_not_grouped merge
#     df_merged = pd.merge(df_grouped, df_not_grouped, how="left", on=["player_name", "last_season"])
    
    return df_grouped
#     return df_merged

In [74]:
app_comp_player_merge(apps_copy, competitions_df, players_df)


yellow_cards  red_cards  goals  assists  \
player_name     last_season                                            
Aaron Boupendza 2019                    0          0      0        0   
Aaron Connolly  2019                    0          0      2        1   
                2020                    0          0      3        2   
                2021                    0          0      0        0   
                2022                    0          0      0        0   
...                                   ...        ...    ...      ...   
İlkay Gündoğan  2019                    7          0      3        2   
                2020                    1          0     14        4   
                2021                    1          0     10        4   
                2022                    1          0      8        5   
                2023                    3          0      7        7   

                             minutes_played  
player_name     last_season                  
Aaron Boupendza 2019                     90  
Aaron Connolly  2019                    878  
                2020                   1219  
                2021                    243  
                2022                     29  
...                                     ...  
İlkay Gündoğan  2019                   1770  
                2020                   2810  
                2021                   2273  
                2022                   2769  
                2023                   2538  

[16620 rows x 5 columns]

In [76]:
app_comp_player_merge(apps_copy, competitions_df, players_df)

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played
0,Aaron Boupendza,2019,0,0,0,0,90
1,Aaron Connolly,2019,0,0,2,1,878
2,Aaron Connolly,2020,0,0,3,2,1219
3,Aaron Connolly,2021,0,0,0,0,243
4,Aaron Connolly,2022,0,0,0,0,29
...,...,...,...,...,...,...,...
16615,İlkay Gündoğan,2019,7,0,3,2,1770
16616,İlkay Gündoğan,2020,1,0,14,4,2810
16617,İlkay Gündoğan,2021,1,0,10,4,2273
16618,İlkay Gündoğan,2022,1,0,8,5,2769


In [66]:
df_merged = app_comp_player_merge(apps_copy, competitions_df, players_df)
df_merged.head(3)

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_club_id,player_current_club_id,...,domestic_league_code,country_of_citizenship,date_of_birth,sub_position,foot,height_in_cm,contract_expiration_date,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,Aaron Boupendza,2019,0,0,0,0,90,427605,3349,7775,...,PO1,Gabon,1996-08-07,Centre-Forward,left,180.0,2025-06-30 00:00:00,Hatayspor Futbol Kulübü,6500000.0,8500000.0
1,Aaron Connolly,2019,0,0,2,1,878,434207,1237,1237,...,GB1,Ireland,2000-01-28,Centre-Forward,right,174.0,2024-06-30 00:00:00,Brighton and Hove Albion Football Club,3500000.0,7000000.0
2,Aaron Connolly,2019,0,0,2,1,878,434207,1237,1237,...,GB1,Ireland,2000-01-28,Centre-Forward,right,174.0,2024-06-30 00:00:00,Brighton and Hove Albion Football Club,3500000.0,7000000.0


In [62]:
df_merged.shape

(315277, 26)

In [64]:
df_merged

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played,game_id,player_id,player_club_id,...,domestic_league_code,country_of_citizenship,date_of_birth,sub_position,foot,height_in_cm,contract_expiration_date,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,Aaron Boupendza,2019,0,0,0,0,90,3229023,427605,3349,...,PO1,Gabon,1996-08-07,Centre-Forward,left,180.0,2025-06-30 00:00:00,Hatayspor Futbol Kulübü,6500000.0,8500000.0
1,Aaron Connolly,2019,0,0,2,1,878,3219422,434207,1237,...,GB1,Ireland,2000-01-28,Centre-Forward,right,174.0,2024-06-30 00:00:00,Brighton and Hove Albion Football Club,3500000.0,7000000.0
2,Aaron Connolly,2019,0,0,2,1,878,3219418,434207,1237,...,GB1,Ireland,2000-01-28,Centre-Forward,right,174.0,2024-06-30 00:00:00,Brighton and Hove Albion Football Club,3500000.0,7000000.0
3,Aaron Connolly,2019,0,0,2,1,878,3219072,434207,1237,...,GB1,Ireland,2000-01-28,Centre-Forward,right,174.0,2024-06-30 00:00:00,Brighton and Hove Albion Football Club,3500000.0,7000000.0
4,Aaron Connolly,2019,0,0,2,1,878,3219407,434207,1237,...,GB1,Ireland,2000-01-28,Centre-Forward,right,174.0,2024-06-30 00:00:00,Brighton and Hove Albion Football Club,3500000.0,7000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315272,İlkay Gündoğan,2023,3,0,7,7,2538,4112818,53622,131,...,ES1,Germany,1990-10-24,Central Midfield,right,180.0,2025-06-30 00:00:00,Futbol Club Barcelona,18000000.0,50000000.0
315273,İlkay Gündoğan,2023,3,0,7,7,2538,4113040,53622,131,...,ES1,Germany,1990-10-24,Central Midfield,right,180.0,2025-06-30 00:00:00,Futbol Club Barcelona,18000000.0,50000000.0
315274,İlkay Gündoğan,2023,3,0,7,7,2538,4112886,53622,131,...,ES1,Germany,1990-10-24,Central Midfield,right,180.0,2025-06-30 00:00:00,Futbol Club Barcelona,18000000.0,50000000.0
315275,İlkay Gündoğan,2023,3,0,7,7,2538,4113180,53622,131,...,ES1,Germany,1990-10-24,Central Midfield,right,180.0,2025-06-30 00:00:00,Futbol Club Barcelona,18000000.0,50000000.0


In [63]:
df_merged.to_csv("data/df_merged.csv", index=False)